In [1]:
## This notebook is for testing trained baseline model on both test and real data

In [1]:
# auto-load when code changes outside
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

import sys
sys.path.insert(0,'..')


In [2]:
# imports 
import os
import random
import spacy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.onnx as onnx
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as T

from src.baseline.noise import Noise
from src.baseline.vocabulary import Vocabulary
from src.utils import print_time, list_of_tensors_to_numpy_arr, plot_image, plot_im
from src.accuracy import WEIGHT_MAP, WEIGHT_MAP_ONLY_SHIELD_COLOR, WEIGHT_MAP_ONLY_CHARGE, WEIGHT_MAP_ONLY_CHARGE_COLOR
from src.baseline.coa_model import get_new_model,load_model, load_model_checkpoint, train_validate_test_split, init_testing_model, train_model, test_model, test_rand_image, get_training_mean_std
from src.baseline.data_loader import get_loader, get_loaders, get_mean, get_std


In [3]:
# seed = 1234
# random.seed(seed)     # python random generator
# np.random.seed(seed)  # numpy random generator

# torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False


In [4]:
!export CUDA_VISIBLE_DEVICES=0
!export CUDA_LAUNCH_BLOCKING=1

In [5]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

device

device(type='cpu')

In [6]:
# torch.cuda.get_device_name(0)

In [7]:
# Hyperparams

embed_size=300
vocab_size = 36
attention_dim=256
# encoder_dim=2048  ### resnet50
encoder_dim=512  ### resnet34 & resnet18
decoder_dim=512
learning_rate = 3e-4
drop_prob=0.3
ignored_idx = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

hyper_params = {'embed_size': embed_size,
                'attention_dim': attention_dim,
                'encoder_dim': encoder_dim,
                'decoder_dim': decoder_dim,
                'vocab_size': vocab_size
              }


## Retrain the baseline model on the real lion images

In [15]:
# data_location =  '/home/salnabulsi/coat-of-arms/data/cropped_coas/out'

data_location =  '/Users/salnabulsi/tub/coat-of-arms/data/cropped_coas/out'

root_folder_images = data_location + '/resized'

caption_file = data_location + '/real_captions_psumsq_lions.txt'
df = pd.read_csv(caption_file)

print("There are {} lion images".format(len(df)))


There are 117 lion images


In [16]:
train_annotation_file = data_location + '/train_captions_psumsq_lions.txt'
val_annotation_file  = data_location + '/val_captions_psumsq_lions.txt'
test_annotation_file  = data_location + '/test_captions_psumsq_lions.txt'

train, validate, test = train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None)

train.to_csv(train_annotation_file, sep=',',index=False)
test.to_csv(test_annotation_file, sep=',',index=False)
validate.to_csv(val_annotation_file, sep=',',index=False)

print("There are {} total images".format(len(df)))

df1 = pd.read_csv(train_annotation_file)
print("There are {} train images".format(len(df1)))

df2 = pd.read_csv(val_annotation_file)
print("There are {} val images".format(len(df2)))

df3 = pd.read_csv(test_annotation_file)
print("There are {} test images".format(len(df3)))


There are 117 total images
There are 70 train images
There are 23 val images
There are 24 test images


In [17]:
# this model is trained on the largest dataset
# model_path = '/home/space/datasets/COA/models/baseline/attention_model-qsub-06.08.2022-12:51:39.pth'
# The trained model has been saved to  /home/space/datasets/COA/experiments/run-10-30-2022-15:43:03/baseline-model-10-30-2022-15:49:09.pth
                
EXP_FOLDER_PATH = '/Users/salnabulsi/tub/coat-of-arms/experiments/'
# EXP_FOLDER_PATH = '/home/space/datasets/COA/experiments/' 
run_path   = EXP_FOLDER_PATH + 'myrunoncluster/'
model_file = 'baseline-model-11-06-2022.pth'


model, optimizer, loss, criterion = load_model(run_path+model_file, 
                                    hyper_params, 
                                    learning_rate,
                                    drop_prob, 
                                    ignored_idx,
                                    pretrained=True)


In [18]:
!export CUDA_LAUNCH_BLOCKING=1
!export CUDA_VISIBLE_DEVICES=2

In [19]:
# BATCH_SIZE = 5
NUM_WORKER = 2 #### this needs multi-core
freq_threshold = 5
batch_size = 5

# 30 minutes to create those, as it's baseline, i ran it several times and it's the same
vocab = Vocabulary(freq_threshold)
vocab.stoi = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'lion': 4, 'rampant': 5, 'passt': 6, 'guard': 7, 'head': 8, 'lions': 9, 'cross': 10, 'moline': 11, 'patonce': 12, 'eagle': 13, 'doubleheaded': 14, 'eagles': 15, 'a': 16, 'b': 17, 'o': 18, 's': 19, 'g': 20, 'e': 21, 'v': 22, '1': 23, '2': 24, '3': 25, '4': 26, '5': 27, '6': 28, '7': 29, '8': 30, '9': 31, '10': 32, '11': 33, 'border': 34, '&': 35}
vocab.itos = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'lion', 5: 'rampant', 6: 'passt', 7: 'guard', 8: 'head', 9: 'lions', 10: 'cross', 11: 'moline', 12: 'patonce', 13: 'eagle', 14: 'doubleheaded', 15: 'eagles', 16: 'a', 17: 'b', 18: 'o', 19: 's', 20: 'g', 21: 'e', 22: 'v', 23: '1', 24: '2', 25: '3', 26: '4', 27: '5', 28: '6', 29: '7', 30: '8', 31: '9', 32: '10', 33: '11', 34: 'border', 35: '&'}

# vocab.stoi = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'lion': 4, 'rampant': 5, 'passt': 6, 'guard': 7, 'head': 8, 'lions': 9, 'a': 16, 'b': 17, 'o': 18, 's': 19, 'g': 20, 'e': 21, 'v': 22, '1': 23, '2': 24, '3': 25, '4': 26, '5': 27, '6': 28, '7': 29, '8': 30, '9': 31, '10': 32, '11': 33, 'border': 34, '&': 35}
# vocab.itos = {0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'lion', 5: 'rampant', 6: 'passt', 7: 'guard', 8: 'head', 9: 'lions', 16: 'a', 17: 'b', 18: 'o', 19: 's', 20: 'g', 21: 'e', 22: 'v', 23: '1', 24: '2', 25: '3', 26: '4', 27: '5', 28: '6', 29: '7', 30: '8', 31: '9', 32: '10', 33: '11', 34: 'border', 35: '&'}


In [20]:
train_loader, train_dataset = get_loader(
    root_folder=root_folder_images,
    annotation_file=train_annotation_file,
    transform=None,  # <=======================
    num_workers=NUM_WORKER,
    vocab=vocab,
    batch_size=batch_size,
    pin_memory=False,
    calc_mean=True
)


In [24]:
mean = get_mean(train_dataset, train_loader, 100 , 100)
std = get_std(train_dataset, train_loader, mean, 100 , 100)

mean, std

tensor(39199804.)
tensor(54382060.)
tensor(-1.3691e+09)
2100000


(tensor(25.8962), tensor(nan))

In [ ]:
transform = T.Compose([
    T.Resize(226),                     
    T.RandomCrop(224),                 
    T.ToTensor(),                               
    T.Normalize(mean, std),
    Noise(0.1, 0.05)
])

print_time('writing the dataloader')

train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset = get_loaders(
    root_folder=root_folder_images,
    train_annotation_file=train_annotation_file,
    val_annotation_file=val_annotation_file,
    test_annotation_file=test_annotation_file,
    transform=transform,
    num_workers=NUM_WORKER,
    vocab=vocab,
    batch_size=batch_size,
    device=device,
    pin_memory=False
)


### Retrain the model


In [30]:
patience = 5
weights_map = WEIGHT_MAP_ONLY_CHARGE
model_folder = run_path
num_epochs = 100
starting_epoch = 0
batch_size = 5

# before training, produces an error when there is a nan in weights
torch.autograd.set_detect_anomaly(True)

model, train_loss, valid_loss, avg_acc, bleu_score = train_model(model, optimizer, criterion, 
                                                                 train_dataset, train_loader, 
                                                                 val_loader, val_dataset, vocab_size,
                                                                 batch_size, patience, num_epochs,
                                                                 device, model_folder, 
                                                                 starting_epoch, weights_map)



Starting training from epoch 0
Current Time =18:29:04 in epoch 0


Epoch 0:   7%| | 1/14 [00:04<01:04,  4.94s/batch, Train loss (

(tensor([[[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
         [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
          nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

Epoch 0:  14%|▏| 2/14 [00:05<00:30,  2.55s/batch, Train loss (

(tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [n

...

(tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [n

Epoch 8: 100%|█| 14/14 [00:16<00:00,  1.16s/batch, Train loss 

validate function called


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/salnabulsi/miniconda/envs/thesis-py38/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/salnabulsi/miniconda/envs/thesis-py38/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/salnabulsi/tub/coat-of-arms/notebooks/../src/baseline/coa_dataset.py", line 7, in <module>
    from src.baseline.vocabulary import Vocabulary
  File "/Users/salnabulsi/tub/coat-of-arms/notebooks/../src/baseline/vocabulary.py", line 1, in <module>
    import spacy
  File "/Users/salnabulsi/miniconda/envs/thesis-py38/lib/python3.8/site-packages/spacy/__init__.py", line 14, in <module>
    from . import pipeline  # noqa: F401
  File "/Users/salnabulsi/miniconda/envs/thesis-py38/lib/python3.8/site-packages/spacy/pipeline/__init__.py", line 1, in <module>
    from .attributeruler import Att

### Init the test model

In [ ]:
test_losses, accuracy_test_list, acc_test_score, test_loss = test_model(model, 
                                                                        criterion,
                                                                        test_loader, 
                                                                        test_dataset, 
                                                                        vocab_size, 
                                                                        device,
                                                                        run_path,
                                                                        False)

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.610000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.360000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like pr

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like pr

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like pr

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.610000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like pr

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.750000

Index 1 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.750000

Index 

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like pr

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like pr

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.610000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 2 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.665000

Index 1 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.750000

Index 1 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.750000

Index 1 does not exist in item, seems like predicted less than Ground truth
Test Acuuracy (in progress): 0.750000

Index 1 does not exist in item, seems like predicted less than Ground truth
Index 

## Visualizing the attentions

In [ ]:
# torch.cuda.empty_cache()
test_rand_image(model,test_dataset,test_loader,device)

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 